In [1]:
from scholars import *
X = np.genfromtxt('data/X_train.txt', delimiter=',')
Y = np.genfromtxt('data/Y_train.txt', delimiter=',')
X_test = np.genfromtxt('data/X_test.txt', delimiter=',')
X, Y = ml.shuffleData(X, Y)
Xtr,Xva,Ytr,Yva = ml.splitData(X,Y,0.75) 
np.random.seed(0)

ImportError: No module named xgboost

In [78]:
#HELPER FUNCTIONS

def stackPredictions(predictions): #TAKES MEAN OF PREDICTIONS AND RETURNS THE FINAL PREDICTION MEAN
    num_data = len(predictions[0])
    finalPrediction = [0]*num_data
    for j in range(num_data):
        finalPrediction[j] = np.mean(np.array(predictions)[:, j])
    return finalPrediction


def convertToFinalPredictions(pred): #CONVERTS ALL PREDICITONS TO 0 AND 1
    final_pred = []
    for i in pred:
        if i>0.5:
            final_pred.append(1)
        else:
            final_pred.append(0)
    return final_pred


def PrintValidationErrorofPredictions(preds): #PRINTS THE ERROR OF EACH PREDICTION ON THE VALIDATION DATA
    i = 0
    for p in preds:
        fp = convertToFinalPredictions(p)
        print(i,"Prediction ERROR: ", 1-sum(fp==Yva)/float(len(Yva)))
        i+=1

def calculateWeight(predictions): #RETURNS THE WEIGHTS FOR EACH PREDICITON
    numOfleaners = len(predictions)
    predictions = np.array([np.array(convertToFinalPredictions(pred)) for pred in predictions])
    weight = np.zeros(numOfleaners)
    for indx, trueClass in enumerate(Yva):
        for i in range(numOfleaners):
            if predictions[i,indx] == trueClass:
                weight[i] += 1
            else:
                weight[i] -= 1
    return weight/sum(weight)


def computeBlendErr(pred,weight): #COMPUTES AND PRINTS THE ERROR OF BLEND
    result = 0
    for i in range(len(pred)):
        result += np.array(pred[i]) * weight[i]
    PrintValidationErrorofPredictions([result])

def computesFinalResult(pred, weights): #COMPUTES BLEND FINAL RESULT AND RETURNS
    result = 0
    for i in range(len(pred)):
        result += np.array(pred[i]) * weights[i]
    return result


def submitPredictions(pred): #PUTS YOUR PREDICTIONS INTO FinalPredictions.txt FOR SUBMISSION
    Y_test = np.vstack((np.arange(X_test.shape[0]), pred)).T
    # Output a file with two columns, a row ID and a confidence in class 1:
    np.savetxt('FinalPredictions.txt', Y_test, '%d, %.2f',header='ID,Predicted', delimiter=',')


def stackResults(predictions): #STACKS PREDICTIONS TO SINGLE ARRAY FOR TRAINING 
    #EACH COLUMN OF PREDICTIONS BECOMES A ROW FOR THE NEW TRAINING
    new_outputs = []
    for pred in predictions:
        new_outputs.append([[j] for j in pred])
    return np.hstack((np.array(new_outputs)))

def finalSubmit(pred_tr, pred_te):
    weights = calculateWeight(pred_tr)
    print("final weights:",weights)
    final_prediction = computesFinalResult(pred_te, weights)
    submitPredictions(final_prediction) 

 

In [4]:
#TRAINS LEARNER
nHidden = 500; 
nnetTR = ml.nnet.nnetClassify(); 
nnetTR.init_weights([Xtr.shape[1],nHidden,2],'random',Xtr,Ytr); 
nnetTR.train(Xtr,Ytr,stopTol=-1,initStep=1.,stopIter=100000)

gradientBoostTR = GradientBoost(Xtr,Ytr)
logistic41featuresTR = LogisticRegression().fit(Xtr[:,:41],Ytr)
adaBoosTR = AdaBoost(Xtr,Ytr, learning_rate=2)
gradientBoost2TR = GradientBoost2(Xtr,Ytr)
randomForestTR = RandomForest(Xtr,Ytr,nFeatures = 50, maxDepth = 15, minLeaf = 4, number_of_learner=25)
randomForest2TR = RandomForest2(Xtr,Ytr)
knn28categoricalTR = ml.knn.knnClassify(Xtr[:,41:69],Ytr)
gradient12Boost2TR = GradientBoost2(Xtr[:,:69], Ytr)
gradient23Boost2TR = GradientBoost2(Xtr[:,41:], Ytr)
newXtr = np.hstack((Xtr[:,:41],Xtr[:,69:]))    
gradient13Boost2TR = GradientBoost2(newXtr, Ytr)

gradientBoost = GradientBoost(X,Y)
logistic41features = LogisticRegression().fit(X[:,:41],Y)
knn28categorical = ml.knn.knnClassify(X[:,41:69],Y)
adaBoost = AdaBoost(X,Y, learning_rate=2)
gradientBoost2 = GradientBoost2(X,Y)
randomForest = RandomForest(X,Y,nFeatures = 50, maxDepth = 15, minLeaf = 4, number_of_learner=25)
randomForest2 = RandomForest2(X,Y)
gradient12Boost2 = GradientBoost2(X[:,:69], Y)
gradient23Boost2 = GradientBoost2(X[:,41:], Y)
newX = np.hstack((X[:,:41],X[:,69:]))    
gradient13Boost2 = GradientBoost2(newX, Y)
nnet = ml.nnet.nnetClassify(); 
nnet.init_weights([Xtr.shape[1],nHidden,2],'random',Xtr,Ytr); 
nnet.train(X,Y,stopTol=-1,initStep=1.,stopIter=100000)


it 2 : Jsur = 0.999101850194, J01 = 0.499550925094
it 4 : Jsur = 1.00808334076, J01 = 0.504041674151
it 8 : Jsur = 1.00751024589, J01 = 0.503682414227
it 16 : Jsur = 1.00660799536, J01 = 0.503323154302
it 32 : Jsur = 0.997863359661, J01 = 0.500089814981
it 64 : Jsur = 1.00456296093, J01 = 0.502425004491
it 128 : Jsur = 0.996552594765, J01 = 0.497754625472
it 256 : Jsur = 0.979810772449, J01 = 0.497395365547
it 512 : Jsur = 0.853949546205, J01 = 0.473684210526
it 1024 : Jsur = 0.914432836267, J01 = 0.505299083887
it 2048 : Jsur = 0.779111293652, J01 = 0.484821268188
it 4096 : Jsur = 0.661029976098, J01 = 0.499550925094
it 8192 : Jsur = 0.593873267594, J01 = 0.466678641997
it 16384 : Jsur = 0.514469240611, J01 = 0.454643434525
it 32768 : Jsur = 0.49778852286, J01 = 0.445302676486
it 65536 : Jsur = 0.508343501148, J01 = 0.457876773846
/opt/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a

In [52]:
#RUNS PREDICTION
gradientPredictionTR = gradientBoostTR.predict(Xva)[:, 1]
adaPredictionTR = adaBoosTR.predict(Xva)[:, 1]
gradientPrediction2TR = gradientBoost2TR.predict(Xva)
forestPredictionTR = randomForestTR.predict(Xva)
forest2PredictionTR = randomForest2TR.predict(Xva)
logistic41PredictionTR = logistic41featuresTR.predict(Xva[:,:41])
knn28PredictionTR = knn28categoricalTR.predict(Xva[:,41:69])
newXva = np.hstack((Xva[:,:41],Xva[:,69:]))
gradient13PredictionTR = gradient13Boost2TR.predict(newXva)  
gradient23PredictionTR = gradient23Boost2TR.predict(Xva[:,41:])
gradient12PredictionTR = gradient12Boost2TR.predict(Xva[:,:69])
nnetPredictionTR = nnet.predict(Xva)


gradientPrediction = gradientBoost.predict(X_test)[:, 1]
adaPrediction = adaBoost.predict(X_test)[:, 1]
gradientPrediction2 = gradientBoost2.predict(X_test)
forestPrediction = randomForest.predict(X_test)
forest2Prediction = randomForest2.predict(X_test)
logistic41Prediction = logistic41features.predict(X_test[:,:41])
knn28Prediction = knn28categorical.predict(X_test[:,41:69])
newXtest = np.hstack((X_test[:,:41],X_test[:,69:]))
gradient13Prediction = gradient13Boost2.predict(newXtest)  
gradient23Prediction = gradient23Boost2.predict(X_test[:,41:])
gradient12Prediction = gradient12Boost2.predict(X_test[:,:69])
nnetPrediction = nnet.predict(X_test)


In [79]:
p_TR = [adaPredictionTR, gradientPredictionTR, forestPredictionTR, nnetPredictionTR, logistic41PredictionTR]
p_TE = [adaPrediction, gradientPrediction, forestPrediction, nnetPrediction, logistic41Prediction]

finalSubmit(p_TR, p_TE)

('final weights:', array([0.09422694, 0.2919708 , 0.46118115, 0.05441274, 0.09820836]))


In [76]:
#BEST WEIGHTS AND LEANERS
# best_weights = [0.15, 0.1,0.3,0.15, 0.3] #BEST
# best_learner_predictions = [adaPrediction,forestPrediction,forest2Prediction,gradientPrediction,gradientPrediction2] #BEST
# final_prediction = computesFinalResult(best_learner_predictions,best_weights)
# submitPredictions(final_prediction)


In [75]:
# all_predictions_TR = [gradientPredictionTR,adaPredictionTR,gradientPrediction2TR ,forestPredictionTR ,forest2PredictionTR ,logistic41PredictionTR,knn28PredictionTR ,gradient13PredictionTR ,gradient23PredictionTR ,gradient12PredictionTR ,nnetPredictionTR]
# all_predictions_TE = [gradientPrediction,adaPrediction,gradientPrediction2 ,forestPrediction ,forest2Prediction ,logistic41Prediction,knn28Prediction,gradient13Prediction,gradient23Prediction,gradient12Prediction,nnetPrediction]


# weights = calculateWeight(all_predictions_TR)
# final_prediction = computesFinalResult(all_predictions_TE, weights)
# submitPredictions(final_prediction)

#PrintValidationErrorofPredictions(all_predictions)

# pred = [forestPredictionTR, forest2PredictionTR, knn28PredictionTR]
# weights = calculateWeight(pred)
# print(weights)
# computeBlendErr(pred, weights)

# best_weights = [0.15, 0.1,0.3,0.15, 0.3] #BEST
# best_learner_predictions_TR = [adaPredictionTR,forestPredictionTR,forest2PredictionTR,gradientPredictionTR,gradientPrediction2TR] #BEST
#computeBlendErr(best_learner_predictions_TR, best_weights)

# final_pred = [forestPrediction, forest2Prediction, knn28Prediction]
# submitPredictions(computesFinalResult(final_pred, weights))

In [ ]:
#stacking outputs
# outputs = [forest2PredictionTR, forestPredictionTR, gradientPrediction2TR, gradientPredictionTR, adaPredictionTR, logistic41PredictionTR, knn28PredictionTR, gradient13PredictionTR, gradient12PredictionTR, gradient23PredictionTR]
#outputs = [forest2PredictionTR, forestPredictionTR, gradientPrediction2TR, gradientPredictionTR, adaPredictionTR]


In [ ]:
stacked_gradient2 = GradientBoost2(xMetaTR, Yva)
stacked_gradient = GradientBoost(xMetaTR, Yva)
stacked_ada = AdaBoost(xMetaTR, Yva)


In [ ]:
stacked_forest2 = RandomForest2(xMetaTR, Yva)
stacked_forest = RandomForest(xMetaTR, Yva, nFeatures = 50, maxDepth = 15, minLeaf = 4, number_of_learner=50)

In [ ]:
stacked_gradient_predict = stacked_gradient.predict(xMeta)[:,1]
stacked_gradient2_predict = stacked_gradient2.predict(xMeta)
stacked_ada_predict = stacked_ada.predict(xMeta)[:, 1]
stacked_forest2_predict = stacked_forest2.predict(xMeta)
stacked_forest_predict = stacked_forest.predict(xMeta)


In [ ]:
#finalPrediction = 0.15*np.array(stacked_ada_predict)+0.1*np.array(stacked_forest_predict)+ 0.3*np.array(stacked_forest2_predict) + 0.15*np.array(stacked_gradient_predict) + 0.3*np.array(stacked_gradient2_predict)

#finalPrediction = 0.25*np.array(stacked_forest_predict)+ 0.25*np.array(stacked_forest2_predict) + 0.25*np.array(stacked_gradient_predict) + 0.25*np.array(stacked_gradient2_predict)




In [ ]:
# final1 = 0.15*np.array(stacked_ada_predict)+0.1*np.array(stacked_forest_predict)+ 0.3*np.array(stacked_forest2_predict) + 0.15*np.array(stacked_gradient_predict) + 0.3*np.array(stacked_gradient2_predict)
# final2 = 0.15*np.array(adaPrediction)+0.1*np.array(forestPrediction)+ 0.3*np.array(forest2Prediction) + 0.15*np.array(gradientPrediction) + 0.3*np.array(gradientPrediction2)
# finalPrediction = 0.4*final1 + 0.6*final2

In [ ]:
Sv = xMetaTR
stack = ml.linearC.linearClassify(Sv,Yva, reg=1e-3)
print "** Stacked AUC: ",stack.auc(Sv,Yva)
Se = xMeta
PeS = stack.predictSoft(Se)
finalPrediction = PeS[:, 1]

In [ ]:
#GRADIENT BOOSTING CLASSIFIER FROM SKLEARN.ENSEMBLE 
#n_estimators = 100


n_estimators = [10,50, 100, 150, 200]
errList = list()
for n in n_estimators:
    gradientBoostingClassifier = GradientBoostingClassifier(n_estimators=n)
    gradientBoostingClassifier.fit(Xtr, Ytr)
    
    prediction = gradientBoostingClassifier.predict_proba(Xva)[:, 1]
    
    errCount = 0
    for pred, va in list(zip(prediction, Yva)):
        if(not ((pred<0.5 and va<0.5) or (pred>0.5 and va>0.5))):
            errCount += 1
    errPercentage = float(errCount)/len(prediction)
    errList.append(errPercentage)
    #print("Prediction: ", prediction)
    print("error percentage: ", errPercentage)
plt.plot(n_estimators, errList)
plt.show()

subsamples = [0.5,0.6,0.7,0.8,0.9,1]
errList = list()
for s in subsamples:
    gradientBoostingClassifier = GradientBoostingClassifier(n_estimators=100, subsample=s)
    gradientBoostingClassifier.fit(Xtr, Ytr)
    
    prediction = gradientBoostingClassifier.predict_proba(Xva)[:, 1]
    
    errCount = 0
    for pred, va in list(zip(prediction, Yva)):
        if(not ((pred<0.5 and va<0.5) or (pred>0.5 and va>0.5))):
            errCount += 1
    errPercentage = float(errCount)/len(prediction)
    errList.append(errPercentage)
    #print("Prediction: ", prediction)
    print("error percentage: ", errPercentage)
plt.plot(subsamples, errList)
plt.show()